# CONTEXT: Great Learning has a an academic support department which receives numerous support requests every day throughout the year. Teams are spread across geographies and try to provide support round the year. Sometimes there are circumstances where due to heavy workload certain request resolutions are delayed, impacting company’s business. Some of the requests are very generic where a proper resolution procedure delivered to the user can solve the problem. Company is looking forward to design an automation which can interact with the user, understand the problem and display the resolution procedure [ if found as a generic request ] or redirect the request to an actual human support executive if the request is complex or not in it’s database

# DATA DESCRIPTION: A sample corpus is attached for your reference. Please enhance/add more data to the corpus using your linguistics skills.

# PROJECT OBJECTIVE: Design a python based interactive semi - rule based chatbot which can do the following:
1. Start chat session with greetings and ask what the user is looking for.
2. Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus.
3. End the chat session only if the user requests to end else ask what the user is looking for. Loop continues till the user asks to end it.
Please use the sample chatbot demo video for reference.

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# Source: https://towardsdatascience.com/how-to-create-a-chatbot-with-python-deep-learning-in-less-than-an-hour-56a063bdfc44
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/gdrive/MyDrive/Colab Notebooks/Project-NLP/GL_Bot.json').read()
intents = json.loads(data_file)

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('/gdrive/MyDrive/Colab Notebooks/Project-NLP/Models/words.pkl','wb'))
pickle.dump(classes,open('/gdrive/MyDrive/Colab Notebooks/Project-NLP/Models/classes.pkl','wb'))

177 documents
8 classes ['Bot', 'Exit', 'Intro', 'NN', 'Olympus', 'Profane', 'SL', 'Ticket']
215 unique lemmatized words [',', 'a', 'able', 'access', 'accessing', 'accuracy', 'activation', 'ada', 'adam', 'aifl', 'aiml', 'am', 'an', 'analysis', 'ann', 'anyone', 'are', 'artificial', 'backward', 'bad', 'bagging', 'batch', 'bayes', 'belong', 'best', 'blended', 'bloody', 'boosting', 'bot', 'buddy', 'can', 'class', 'classification', 'classifier', 'completely', 'component', 'confusion', 'contact', 'crazy', 'create', 'cross', 'customer', 'cya', 'day', 'deep', 'dendogram', 'dense', 'did', 'diffult', 'dnn', 'do', 'ensemble', 'epoch', 'evening', 'explain', 'extreme', 'f***', 'facing', 'find', 'first', 'fl', 'for', 'forest', 'forward', 'from', 'function', 'good', 'goodbye', 'gradient', 'great', 'hate', 'have', 'hell', 'hello', 'help', 'helped', 'hey', 'hi', 'hidden', 'hour', 'how', 'hyper', 'i', 'idiot', 'imputer', 'in', 'intelligence', 'is', 'issue', 'it', 'itertations', 'jerk', 'joke', 'kfold', 

# Building the Model

In [9]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('/gdrive/MyDrive/Colab Notebooks/Project-NLP/Models/chatbot_model.h5', hist)

print("model created")

Epoch 1/200
36/36 [==============================] - 1s 1ms/step - loss: 2.0245 - accuracy: 0.2480
Epoch 2/200
36/36 [==============================] - 0s 1ms/step - loss: 1.9586 - accuracy: 0.2456
Epoch 3/200
36/36 [==============================] - 0s 1ms/step - loss: 1.7660 - accuracy: 0.3395
Epoch 4/200
36/36 [==============================] - 0s 1ms/step - loss: 1.7197 - accuracy: 0.4075
Epoch 5/200
36/36 [==============================] - 0s 1ms/step - loss: 1.5511 - accuracy: 0.3879
Epoch 6/200
36/36 [==============================] - 0s 1ms/step - loss: 1.5037 - accuracy: 0.4453
Epoch 7/200
36/36 [==============================] - 0s 2ms/step - loss: 1.2480 - accuracy: 0.5797
Epoch 8/200
36/36 [==============================] - 0s 2ms/step - loss: 1.2172 - accuracy: 0.5669
Epoch 9/200
36/36 [==============================] - 0s 1ms/step - loss: 1.1732 - accuracy: 0.5673
Epoch 10/200
36/36 [==============================] - 0s 1ms/step - loss: 0.9919 - accuracy: 0.7148
Epoch 11/